In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from __future__ import division
import missingno as msno
import xgboost
import numpy as np
import xgboost
from sklearn.model_selection import StratifiedKFold, GridSearchCV

/home/pranay/tensorflow/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from imblearn.over_sampling import SMOTE,RandomOverSampler

In [3]:
from utils import *

In [4]:
pd.set_option('display.max_colwidth', -1)

In [129]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [62]:
test = pd.read_csv('application_test.csv')

In [63]:
print 'Reading train ...'
train = pd.read_csv('application_train.csv')
print 'Shape of train is %s'%(str(train.shape))
bureau_balance = pd.read_csv('bureau_balance.csv')
print 'The shape of bureau_balance is %s'%(str(bureau_balance.shape))
credit_card_balance = pd.read_csv('credit_card_balance.csv')
print 'The shape of creditcard_balance is %s'%(str(credit_card_balance.shape))
installments_payments = pd.read_csv('installments_payments.csv')
print 'The shape of installments_payments is %s'%(str(installments_payments.shape))
pos_cash_balance = pd.read_csv('POS_CASH_balance.csv')
print 'The shape of pos_cash_balance is %s'%(str(pos_cash_balance.shape))
previous_application = pd.read_csv('previous_application.csv')
print 'The shape of previous_application is %s'%(str(previous_application.shape))
descriptions = pd.read_csv('HomeCredit_columns_description.csv')
print 'The shape of descriptions is %s'%(str(descriptions.shape))
bureau = pd.read_csv('bureau.csv')
print 'The shape of bureau is %s'%(str(bureau.shape))
print 'Done!'

Reading train ...
Shape of train is (307511, 122)
The shape of bureau_balance is (27299925, 3)
The shape of creditcard_balance is (3840312, 23)
The shape of installments_payments is (13605401, 8)
The shape of pos_cash_balance is (10001358, 8)
The shape of previous_application is (1670214, 37)
The shape of descriptions is (219, 5)
The shape of bureau is (1716428, 17)
Done!


In [140]:
my_utils = MyUtils(previous_application)
my_utils.getColDesc()

AMT_ANNUITY ['Loan annuity' 'Annuity of the Credit Bureau credit'
 'Annuity of previous application']
AMT_APPLICATION ['For how much credit did client ask on the previous application']
AMT_DOWN_PAYMENT ['Down payment on the previous application']
FLAG_LAST_APPL_PER_CONTRACT ['Flag if it was last application for the previous contract. Sometimes by mistake of client or our clerk there could be more applications for one single contract']
NFLAG_LAST_APPL_IN_DAY ['Flag if the application was the last application per day of the client. Sometimes clients apply for more applications a day. Rarely it could also be error in our system that one application is in the database twice']
RATE_DOWN_PAYMENT ['Down payment rate normalized on previous credit']
RATE_INTEREST_PRIMARY ['Interest rate normalized on previous credit']
RATE_INTEREST_PRIVILEGED ['Interest rate normalized on previous credit']
NAME_CASH_LOAN_PURPOSE ['Purpose of the cash loan']
DAYS_DECISION ['Relative to current application when w

In [139]:
previous_application.sort_values('SK_ID_CURR').head()

,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_DOWN_PAYMENT,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,HOUR_APPR_PROCESS_START_PREV,NAME_TYPE_SUITE_PREV,WEEKDAY_APPR_PROCESS_START_PREV,NAME_CONTRACT_TYPE_PREV,AMT_CREDIT_PREV,AMT_GOODS_PRICE_PREV,NAME_CONTRACT_STATUS_PREV,app_cred_diff
201668,1369693,100001,3951.000,24835.500,2520.000,Y,1,0.104,nan,nan,XAP,-1740,Cash through the bank,XAP,Refreshed,Mobile,POS,XNA,Country-wide,23,Connectivity,8.000,high,POS mobile with interest,365243.000,-1709.000,-1499.000,-1619.000,-1612.000,0.000,13,Family,FRIDAY,Consumer loans,23787.000,24835.500,Approved,1048.500
892077,1038818,100002,9251.775,179055.000,0.000,Y,1,0.000,nan,nan,XAP,-606,XNA,XAP,New,Vehicles,POS,XNA,Stone,500,Auto technology,24.000,low_normal,POS other with interest,365243.000,-565.000,125.000,-25.000,-17.000,0.000,9,NaN,SATURDAY,Consumer loans,179055.000,179055.000,Approved,0.000
575941,1810518,100003,98356.995,900000.000,nan,Y,1,nan,nan,nan,XNA,-746,XNA,XAP,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.000,low_normal,Cash X-Sell: low,365243.000,-716.000,-386.000,-536.000,-527.000,1.000,12,Unaccompanied,FRIDAY,Cash loans,1035882.000,900000.000,Approved,-135882.000
1021650,2636178,100003,64567.665,337500.000,0.000,Y,1,0.000,nan,nan,XAP,-828,Cash through the bank,XAP,Refreshed,Furniture,POS,XNA,Stone,1400,Furniture,6.000,middle,POS industry with interest,365243.000,-797.000,-647.000,-647.000,-639.000,0.000,17,Family,SUNDAY,Consumer loans,348637.500,337500.000,Approved,-11137.500
1223745,2396755,100003,6737.310,68809.500,6885.000,Y,1,0.100,nan,nan,XAP,-2341,Cash through the bank,XAP,Refreshed,Consumer Electronics,POS,XNA,Country-wide,200,Consumer electronics,12.000,middle,POS household with interest,365243.000,-2310.000,-1980.000,-1980.000,-1976.000,1.000,15,Family,SATURDAY,Consumer loans,68053.500,68809.500,Approved,756.000


In [150]:
previous_application.groupby('SK_ID_CURR').NAME_CLIENT_TYPE

AttributeError: Cannot access callable attribute 'mode' of 'SeriesGroupBy' objects, try using the 'apply' method

In [131]:
pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,MONTHS_BALANCE_POS,SK_DPD_DEF_POS,NAME_CONTRACT_STATUS_POS
0,1803195,182943,48.000,45.000,0,-31,0,Active
1,1715348,367990,36.000,35.000,0,-33,0,Active
2,1784872,397406,12.000,9.000,0,-32,0,Active
3,1903291,269225,48.000,42.000,0,-35,0,Active
4,2341044,334279,36.000,35.000,0,-35,0,Active


In [132]:
previous_application[previous_application.SK_ID_PREV == 1803195]

,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_DOWN_PAYMENT,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,HOUR_APPR_PROCESS_START_PREV,NAME_TYPE_SUITE_PREV,WEEKDAY_APPR_PROCESS_START_PREV,NAME_CONTRACT_TYPE_PREV,AMT_CREDIT_PREV,AMT_GOODS_PRICE_PREV,NAME_CONTRACT_STATUS_PREV,app_cred_diff
1650130,1803195,182943,22629.600,630000.000,nan,Y,1,nan,nan,nan,XNA,-1035,Cash through the bank,XAP,Repeater,XNA,Cash,x-sell,AP+ (Cash loan),5,XNA,48.000,low_normal,Cash X-Sell: low,365243.000,-1005.000,405.000,365243.000,365243.000,0.000,18,Unaccompanied,WEDNESDAY,Cash loans,630000.000,630000.000,Approved,0.000


In [ ]:
pos_cash_balance.SK_ID_CURR.value_counts()

In [122]:
concatted.shape

(98394, 292)

In [16]:
corr_train = train.corr()

In [34]:
corr_test = test.corr()

In [ ]:
corr_train[corr_train > 0.70][50:]

In [ ]:
corr_test[corr_test > 0.70][:50]

In [31]:
corr_train[corr_train == 0.950842]

Index([u'SK_ID_CURR', u'TARGET', u'CNT_CHILDREN', u'AMT_INCOME_TOTAL',
       u'AMT_CREDIT', u'AMT_ANNUITY', u'AMT_GOODS_PRICE',
       u'REGION_POPULATION_RELATIVE', u'DAYS_BIRTH', u'DAYS_EMPLOYED',
       ...
       u'FLAG_DOCUMENT_18', u'FLAG_DOCUMENT_19', u'FLAG_DOCUMENT_20',
       u'FLAG_DOCUMENT_21', u'AMT_REQ_CREDIT_BUREAU_HOUR',
       u'AMT_REQ_CREDIT_BUREAU_DAY', u'AMT_REQ_CREDIT_BUREAU_WEEK',
       u'AMT_REQ_CREDIT_BUREAU_MON', u'AMT_REQ_CREDIT_BUREAU_QRT',
       u'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=106)

<strong>Remember to remove one of these columns in credit_card_application as they are correlated</strong>
7. CNT_DRAWINGS_ATM_CURRENT | CNT_DRAWINGS_CURRENT columns are correlated (1.0)
8. AMT_DRAWINGS_ATM_CURRENT | AMT_DRAWINGS_CURRENT are correlated(1.0)
9. SK_DPD | SK_DPD_DEF are correlated by 1.0

<strong>These are correlated in installments_payments</strong>
1. DAYS_INSTALLMENT | DAYS_ENTRY_PAYMENT (0.99)
2. AMT_INSTALMENT | AMT_PAYMENT

<strong>In pos_cash_balance</strong>
1. CNT_INSTALMENT | CNT_INSTALMENT_FUTURE (0.87)

<strong>In train</strong>
1. AMT_CREDIT | AMT_ANNUITY (0.77)
2. AMT_CREDIT | AMT_GOODS_PRICE (0.98)
3. AMT_ANNUITY | AMT_GOODS_PRICE (0.77)
4. CNT_FAM_MEMBERS | CNT_CHILDREN
5. REGION_RATING_CLIENT | REGION_RATING_CLIENT_W_CITY (0.95)

#### Let's drop correlated variables

In [39]:
test = test.drop(['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'CNT_CHILDREN', 'REGION_RATING_CLIENT_W_CITY'], axis=1)

### Lets try to calculate mix of loans

In [ ]:
train.head()

In [ ]:
credit_card_balance.sort_values('SK_ID_CURR').head(15)

In [ ]:
my_utils = MyUtils(train)
my_utils.getColDesc()

In [21]:
previous_application.SK_ID_PREV.unique().shape

(1670214,)

In [29]:
train.shape

(307511, 122)

In [30]:
test.shape

(48744, 121)

In [22]:
previous_application.shape

(1670214, 37)

In [23]:
credit_card_balance.SK_ID_PREV.unique().shape

(104307,)

In [24]:
pos_cash_balance.SK_ID_PREV.unique().shape

(936325,)

In [ ]:
pos_cash_balance.head()

In [ ]:
credit_card_balance.sort_values('SK_ID_PREV').head(15)

In [ ]:
previous_application[previous_application.SK_ID_PREV == 1803195]

In [ ]:
test[test.SK_ID_CURR == 131335]

In [35]:
my_utils = MyUtils(train)
nonnulls = my_utils.getNonnullCount()

In [ ]:
my_utils = MyUtils(credit_card_balance)
my_utils.getDfOfMax().sort_values('SK_ID_PREV')
corr_df = my_utils.getDfOfMax()[my_utils.getDfOfMax().SK_ID_PREV == 2243411].sort_values('MONTHS_BALANCE').corr()
corr_df[corr_df == 1]

In [ ]:
y = np.bincount(credit_card_balance.SK_ID_PREV.value_counts().values)
ii = np.nonzero(y)[0]
zip(ii,y[ii])

In [ ]:
train_skids = train.SK_ID_CURR
test_skids = test.SK_ID_CURR
appended = train_skids.append(test_skids)
assert len(appended) == len(train_skids) + len(test_skids)
not_in_prev = [ skid for skid in appended if skid not in  previous_application.SK_ID_CURR.unique()]
len(not_in_prev)
not_in_prev_df = pd.DataFrame(not_in_prev)
not_in_prev_df.columns = ['SK_ID_CURR']
# not_in_prev_df.SK_ID_CURR.value_counts()

# len(appended)
# previous_application.SK_ID_CURR.unique()

### Renaming colums which are similar
Not doing this is causing problems in xgboost. Naming is by adding dataframe name at the end

In [64]:
credit_card_balance['NAME_CONTRACT_STATUS_CRED'] = credit_card_balance.NAME_CONTRACT_STATUS
credit_card_balance = credit_card_balance.drop(['NAME_CONTRACT_STATUS'], axis=1)
credit_card_balance['MONTHS_BALANCE_CRED'] = credit_card_balance.MONTHS_BALANCE
credit_card_balance = credit_card_balance.drop(['MONTHS_BALANCE'], axis=1)

bureau_balance['MONTHS_BALANCE_BUR'] = bureau_balance.MONTHS_BALANCE
bureau_balance = bureau_balance.drop(['MONTHS_BALANCE'], axis=1)
pos_cash_balance['MONTHS_BALANCE_POS'] = pos_cash_balance.MONTHS_BALANCE
pos_cash_balance = pos_cash_balance.drop(['MONTHS_BALANCE'], axis=1)
pos_cash_balance['SK_DPD_DEF_POS'] = pos_cash_balance.SK_DPD_DEF
pos_cash_balance = pos_cash_balance.drop(['SK_DPD_DEF'], axis=1)
pos_cash_balance['NAME_CONTRACT_STATUS_POS'] = pos_cash_balance.NAME_CONTRACT_STATUS
pos_cash_balance = pos_cash_balance.drop(['NAME_CONTRACT_STATUS'], axis=1)


previous_application['HOUR_APPR_PROCESS_START_PREV'] = previous_application.HOUR_APPR_PROCESS_START
previous_application = previous_application.drop(['HOUR_APPR_PROCESS_START'], axis=1)
previous_application['NAME_TYPE_SUITE_PREV'] = previous_application.NAME_TYPE_SUITE
previous_application = previous_application.drop(['NAME_TYPE_SUITE'], axis=1)
previous_application['WEEKDAY_APPR_PROCESS_START_PREV'] = previous_application.WEEKDAY_APPR_PROCESS_START
previous_application = previous_application.drop(['WEEKDAY_APPR_PROCESS_START'], axis=1)
previous_application['NAME_CONTRACT_TYPE_PREV'] = previous_application.NAME_CONTRACT_TYPE
previous_application = previous_application.drop(['NAME_CONTRACT_TYPE'], axis=1)
previous_application['AMT_CREDIT_PREV'] = previous_application.AMT_CREDIT
previous_application = previous_application.drop(['AMT_CREDIT'], axis=1)
previous_application['AMT_GOODS_PRICE_PREV'] = previous_application.AMT_GOODS_PRICE
previous_application = previous_application.drop(['AMT_GOODS_PRICE'], axis=1)
previous_application['NAME_CONTRACT_STATUS_PREV'] = previous_application.NAME_CONTRACT_STATUS
previous_application= previous_application.drop(['NAME_CONTRACT_STATUS'], axis=1)

### Starting to create feature columns
Columns with Levels needs to be unstacked after group by
<p><strong>This is to get pure columns. It is almost like creating dummies</strong></p>

In [65]:
my_utils = MyUtils(pos_cash_balance)
sk_dpd_def_pos_df = my_utils.groupBySkidMean('SK_DPD_DEF_POS')
sk_dpd_df = my_utils.groupBySkidMean('SK_DPD')

# sk_dpd_def_pos_df.merge(sk_dpd_df, how='left', on='SK_ID_CURR').corr()

bur_grp_df =bureau.groupby('SK_ID_CURR').CREDIT_ACTIVE.value_counts().unstack()
bur_grp_df.columns = ['credit_active', 'credit_bad_debt', 'credit_closed', 'credit_sold']
bur_grp_df = bur_grp_df.fillna(value=0).reset_index()

bur_curr_df = bureau.groupby('SK_ID_CURR').CREDIT_CURRENCY.value_counts().unstack()
bur_curr_df.columns = ['currency1', 'currency2', 'currency3', 'currency4']
bur_curr_df = bur_curr_df.fillna(value=0).reset_index()

bur_credtype_df = bureau.groupby('SK_ID_CURR').CREDIT_TYPE.value_counts().unstack()
# bur_credtype_df.columns = ['currency1', 'currency2', 'currency3', 'currency4']
bur_credtype_df = bur_credtype_df.fillna(value=0).reset_index()

my_utils = MyUtils(bureau)
bur_amtcredsum_df = my_utils.groupBySkidMean('AMT_CREDIT_SUM')


In [66]:
my_utils = MyUtils(bureau)
bur_dayscredupdate_df = my_utils.groupBySkidMean('DAYS_CREDIT_UPDATE')
bur_creddayoverdue_df = my_utils.groupBySkidMean('CREDIT_DAY_OVERDUE')

bur_amt_cred_sum_overdue_df = my_utils.groupBySkidMean('AMT_CREDIT_SUM_OVERDUE')

bur_amt_cred_sum_limit_df = my_utils.groupBySkidMean('AMT_CREDIT_SUM_LIMIT')

bur_amt_credit_sum_debt_df = my_utils.groupBySkidMean('AMT_CREDIT_SUM_DEBT')

In [67]:
bb_cnt_df = bureau_balance.SK_ID_BUREAU.value_counts().to_frame().reset_index()
bb_cnt_df.columns = ['SK_ID_BUREAU', 'bb_cnt']

bureau_bb_cnt_df = bureau.merge(bb_cnt_df, on='SK_ID_BUREAU', how='inner')
grp_cnt_bureau_df = bureau_bb_cnt_df.groupby('SK_ID_CURR').bb_cnt.sum().to_frame().reset_index()

In [68]:
bureau_cnt_df = bureau.SK_ID_CURR.value_counts().to_frame().reset_index()
bureau_cnt_df.columns = ['SK_ID_CURR', 'bureau_cnt']

In [69]:
my_utils = MyUtils(bureau)
bureau_cnt_df = my_utils.skidCount('bureau_')

In [70]:
counts = bureau_balance['SK_ID_BUREAU'].value_counts().reset_index()
burbal_df = pd.DataFrame(counts)
burbal_df.columns = ['SK_ID_BUREAU',  'bureau_cnt']

In [71]:
bureau[['SK_ID_CURR', 'SK_ID_BUREAU']].merge(burbal_df, on='SK_ID_BUREAU', how='left') \
            .bureau_cnt.isnull().value_counts()

True     942074
False    774354
Name: bureau_cnt, dtype: int64

### Merging Bureau and Bureau balance
Remember that each one sk_id_curr is asoociated with many sk_id_bureau which in turn has monthly records so will have to multiply both

In [72]:
#merge bureau and bureau_balance
bureau_burbal = bureau[['SK_ID_CURR', 'SK_ID_BUREAU']].merge(burbal_df, on='SK_ID_BUREAU', how='left')
bureau_burbal.bureau_cnt = bureau_burbal.bureau_cnt.fillna(value=0)
bureau_burbal_df = bureau_burbal.groupby('SK_ID_CURR').bureau_cnt.sum().reset_index()

### Count of sk_id_curr in previous applications

In [73]:
prev_app_cnt = previous_application.SK_ID_CURR.value_counts().reset_index()
prev_app_cnt_df = pd.DataFrame(prev_app_cnt)
prev_app_cnt_df.columns = ['SK_ID_CURR', 'prev_app_cnt']

### Count of skidcurr in credit card balance

In [74]:
cred_cnt = credit_card_balance.SK_ID_CURR.value_counts().reset_index()
cred_cnt_df = pd.DataFrame(cred_cnt)
cred_cnt_df.columns = ['SK_ID_CURR', 'cred_cnt']

In [75]:
my_utils = MyUtils(df=credit_card_balance)
amt_balance_df = my_utils.groupBySkidMean('AMT_BALANCE')

In [76]:
amt_balance_df.head()

,SK_ID_CURR,AMT_BALANCE
0,100006,0.000000
1,100011,54482.111149
2,100013,18159.919219
3,100021,0.000000
4,100023,0.000000


In [77]:
#name_contr_df
nam_contr = previous_application[['SK_ID_CURR', 'NAME_CONTRACT_TYPE_PREV']]
nam_cntr_dummied_df = nam_contr.groupby('SK_ID_CURR').NAME_CONTRACT_TYPE_PREV.value_counts().unstack()
nam_cntr_dummied_df.columns = ['cash_loans', 'consumer_loans', 'revolving_loans', 'xna']
nam_cntr_dummied_df = nam_cntr_dummied_df.fillna(value=0).reset_index()

In [78]:
my_utils = MyUtils(previous_application)
cnt_payment_df = my_utils.groupBySkidMean('CNT_PAYMENT')

In [79]:
my_utils = MyUtils(previous_application)
amt_goods_price_df = my_utils.groupBySkidMean('AMT_GOODS_PRICE_PREV')
amt_annuity_df = my_utils.groupBySkidMean('AMT_ANNUITY')

previous_application['app_cred_diff'] = previous_application.AMT_APPLICATION - previous_application.AMT_CREDIT_PREV
app_cred_diff_df = my_utils.groupBySkidMean('app_cred_diff')
amt_credit_df = my_utils.groupBySkidMean('AMT_CREDIT_PREV')

In [80]:
cols_select = ['SK_ID_CURR', 'NAME_CONTRACT_STATUS_PREV']
prevapp_select = previous_application[cols_select]

In [81]:
prev_name_cntr_status_df = prevapp_select.groupby('SK_ID_CURR').NAME_CONTRACT_STATUS_PREV.value_counts().unstack().fillna(value=0)
prev_name_cntr_status_df.columns = ['NAME_CONTRACT_STATUS_PREV_approved', 'NAME_CONTRACT_STATUS_PREV_canceled', 'NAME_CONTRACT_STATUS_PREV_refused', 'NAME_CONTRACT_STATUS_PREV_unused']
prev_name_cntr_status_df = prev_name_cntr_status_df.reset_index()

In [82]:
my_utils = MyUtils(pos_cash_balance)
count_contract_status_df = my_utils.groupBySkidCount('NAME_CONTRACT_STATUS_POS')
future_df = my_utils.groupBySkidCount('CNT_INSTALMENT_FUTURE')

In [83]:
installments_payments['diff_amount'] = installments_payments.AMT_INSTALMENT - installments_payments.AMT_PAYMENT
installments_payments['diff_days'] = installments_payments.DAYS_INSTALMENT - installments_payments.DAYS_ENTRY_PAYMENT
my_utils = MyUtils(installments_payments)
diff_days_mean_df = my_utils.groupBySkidMean('diff_days')
diff_amt_mean_df = my_utils.groupBySkidMean('diff_amount')

### Start the Main Process

In [84]:
train_x = train.drop(['TARGET'], axis=1)

In [85]:
assert train_x.shape[1] == test.shape[1]

In [86]:
concatted = pd.concat([train_x, test])

### Let's remove correlated columns here

In [87]:
concatted = concatted.drop(['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'CNT_CHILDREN', 'REGION_RATING_CLIENT_W_CITY'], axis=1)

In [88]:
assert installments_payments[installments_payments.diff_amount.isnull()].shape[0] == installments_payments[installments_payments.diff_days.isnull()].shape[0]

In [89]:
#Since they share same rows, joint them
join_amt_days = diff_amt_mean_df.merge(diff_days_mean_df, on='SK_ID_CURR', how='inner')

In [90]:
prev_app_cnt_df.shape

(338857, 2)

In [91]:
cred_cnt_df.shape

(103558, 2)

### Doing Joins with Engineered Features

In [96]:
concat_with_features = concatted.merge(future_df, on='SK_ID_CURR', how='left')  \
            .merge(join_amt_days, on='SK_ID_CURR', how='left') \
            .merge(count_contract_status_df, on='SK_ID_CURR', how='left') \
            .merge(app_cred_diff_df, on='SK_ID_CURR', how='left') \
            .merge(amt_credit_df, on='SK_ID_CURR', how='left') \
            .merge(amt_annuity_df, on='SK_ID_CURR', how='left') \
            .merge(amt_goods_price_df, on='SK_ID_CURR', how='left') \
            .merge(amt_balance_df, on='SK_ID_CURR', how='left') \
            .merge(cnt_payment_df, on='SK_ID_CURR', how='left') \
            .merge(prev_app_cnt_df, on='SK_ID_CURR', how='left') \
            .merge(cred_cnt_df, on='SK_ID_CURR', how='left') \
            .merge(bureau_burbal_df, on='SK_ID_CURR', how='left') \
            .merge(bur_grp_df, on='SK_ID_CURR', how='left') \
            .merge(bur_curr_df, on='SK_ID_CURR', how='left') \
            .merge(bur_credtype_df, on='SK_ID_CURR', how='left') \
            .merge(bur_amtcredsum_df, on='SK_ID_CURR', how='left') \
            .merge(bur_dayscredupdate_df, on='SK_ID_CURR', how='left') \
            .merge(bur_creddayoverdue_df, on='SK_ID_CURR', how='left') \
            .merge(nam_cntr_dummied_df, on='SK_ID_CURR', how='left') \
            .merge(prev_name_cntr_status_df, on='SK_ID_CURR', how='left') \
            .merge(sk_dpd_def_pos_df, on='SK_ID_CURR', how='left') \
            .merge(sk_dpd_df, on='SK_ID_CURR', how='left') \
            .merge(bur_amt_cred_sum_overdue_df, on='SK_ID_CURR', how='left') \
            .merge(bur_amt_cred_sum_limit_df, on='SK_ID_CURR', how='left') \
            .merge(bur_amt_credit_sum_debt_df, on='SK_ID_CURR', how='left')

In [97]:
concat_with_features.iloc[:,129:].head()

,bureau_cnt,credit_active,credit_bad_debt,credit_closed,credit_sold,currency1,currency2,currency3,currency4,Another type of loan,Car loan,Cash loan (non-earmarked),Consumer credit,Credit card,Interbank credit,Loan for business development,Loan for purchase of shares (margin lending),Loan for the purchase of equipment,Loan for working capital replenishment,Microloan,Mobile operator loan,Mortgage,Real estate loan,Unknown type of loan,AMT_CREDIT_SUM,DAYS_CREDIT_UPDATE,CREDIT_DAY_OVERDUE,cash_loans,consumer_loans,revolving_loans,xna,NAME_CONTRACT_STATUS_PREV_approved,NAME_CONTRACT_STATUS_PREV_canceled,NAME_CONTRACT_STATUS_PREV_refused,NAME_CONTRACT_STATUS_PREV_unused,SK_DPD_DEF_POS,SK_DPD,AMT_CREDIT_SUM_OVERDUE,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_DEBT
0,110.0,2.0,0.0,6.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,108131.945625,-499.875,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7997.14125,49156.2
1,0.0,1.0,0.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,254350.125000,-816.000,0.0,1.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,202500.00000,0.0
2,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,94518.900000,-532.000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,2.0,2.0,0.0,5.0,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,146250.000000,-783.000,0.0,4.0,2.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0


In [98]:
concat_with_features.columns.get_loc('AMT_BALANCE')

125

In [99]:
#fill nas after 138th column with zeros. lower cased column names are engineered columns
concat_with_features.iloc[:,125:] = concat_with_features.iloc[:,125:].fillna(value=0)

In [100]:
y_train = train.TARGET

### Load Imputer Libraries

In [101]:
from sklearn.preprocessing import Imputer
my_imputer = Imputer()
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
clf = RandomForestClassifier()
# clf = LogisticRegression()

In [102]:
concat_with_features_drop = concat_with_features.drop('SK_ID_CURR', axis=1)

In [103]:
dummied = pd.get_dummies(concat_with_features_drop)

In [104]:
imputed = my_imputer.fit_transform(dummied)

In [105]:
imputed_df = pd.DataFrame(imputed)
imputed_df.columns = dummied.columns

### Split the test and train back

In [106]:
new_x_train_df = imputed_df[:train_x.shape[0]]
new_test_df = imputed_df[train_x.shape[0]:]

In [107]:
assert new_x_train_df.shape[0] == train.shape[0] 

### Initiate XGBoost Classifier

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier()
n_estimators = [50, 100, 150, 200]
max_depth = [2, 4, 6, 8]
param_grid = dict(max_depth=max_depth, n_estimators=n_estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, cv=kfold, n_jobs=-1)
grid_result = grid_search.fit(new_x_train_df, y_train)
print 'Best %s using %s'%(str(grid_result.best_score_), str(grid_result.best_params_))

In [108]:
new_x_train_df['TARGET'] = train.TARGET

/home/pranay/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [109]:

train_zeros = new_x_train_df[new_x_train_df.TARGET == 0]
train_ones = new_x_train_df[new_x_train_df.TARGET == 1]
sampled_indices = np.random.choice(train_zeros.index, size=[11, train_ones.shape[0]], replace=False)
print 'shape of sampled indices is %s'%(str(sampled_indices.shape))
test_dropid = new_test_df
ensembled = np.zeros([11, test.shape[0]])
preds_list = []
for i in range(11):
    print 'starting %s th sample'%(str(i))
    new_train = pd.concat([train_zeros.loc[sampled_indices[i]], train_ones])
    train_y = new_train.TARGET
    train_x = new_train.drop(['TARGET'], axis=1)
    
    concatted = pd.concat([train_x, test_dropid])
    concatted_dummied = pd.get_dummies(concatted)
    imputed_concatted = my_imputer.fit_transform(concatted_dummied)
    imputed_concatted_df = pd.DataFrame(imputed_concatted)
    imputed_concatted_df.columns = concatted_dummied.columns
    #split them back to train and test
    train_x_after_impute = imputed_concatted_df[:train_x.shape[0]]
    test_x = imputed_concatted_df[train_x.shape[0]:]
    clf.fit(train_x_after_impute, train_y)
    preds = clf.predict_proba(test_x)
    pred_actual = preds[:,1]
    ensembled[i] = pred_actual



shape of sampled indices is (11, 24825)
starting 0 th sample
starting 1 th sample
starting 2 th sample
starting 3 th sample
starting 4 th sample
starting 5 th sample
starting 6 th sample
starting 7 th sample
starting 8 th sample
starting 9 th sample
starting 10 th sample


In [78]:
dropped = new_x_train_df.drop(['TARGET'], axis=1)
x_resampled, y_resampled = SMOTE().fit_sample(dropped, y_train)
clf.fit(x_resampled, y_resampled)
predicts = clf.predict_log_proba(new_test_df)

/home/pranay/tensorflow/local/lib/python2.7/site-packages/sklearn/ensemble/forest.py:623: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)


In [104]:
predicts = clf.predict_proba(new_test_df)

In [100]:
y_train.shape

(307511,)

In [106]:
meaned = predicts[:,0]

In [110]:
meaned = np.mean(ensembled, axis=0)
assert meaned.shape[0] == 48744

In [ ]:
clf.fit(x_resampled, y_resampled)

In [203]:
preds = clf.predict_proba(new_test)
pred_actual = preds[:,1]

In [ ]:
assert len(not_included) == test.shape[0] - test_merged.shape[0]

In [111]:
preds_df = pd.DataFrame(meaned)
subm = pd.concat([test.SK_ID_CURR, preds_df], axis=1)
subm.columns = ['SK_ID_CURR', 'TARGET']

In [112]:
subm_csv = subm.to_csv('submission.csv', sep=',', index=False)

In [113]:
%mv submission.csv /home/pranay/Downloads/

In [266]:
credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [ ]:
pos_cash_balance.SK_ID_CURR.value_counts()

In [ ]:
credit_card_balance.SK_ID_CURR.value_counts()

In [ ]:
previous_application.SK_ID_CURR.value_counts()

In [165]:
for col in credit_card_balance

float64    15
int64      7 
object     1 
dtype: int64

In [167]:
credit_card_balance.SK_ID_CURR.dtype == 'in'

True

In [227]:
assert new_train.shape[0] == train_ones.shape[0] * 2

In [147]:
floats = less_nulls_concat.loc[:,less_nulls_concat.dtypes == 'float64']
ints = less_nulls_concat.loc[:,less_nulls_concat.dtypes == 'int64']
objectly = less_nulls_concat.loc[:,less_nulls_concat.dtypes == 'object']

In [94]:
floats = floats.fillna(floats.mean())
ints = ints.fillna(ints.mean())

In [101]:
assert floats.shape[0] == concatted.shape[0]

In [ ]:
get_nulls(ints)

In [243]:
pred_actual_df = pd.DataFrame(pred_actual)

In [244]:
subm = pd.concat([test.SK_ID_CURR, pred_actual_df], axis=1)

In [245]:
subm.columns = ['SK_ID_CURR', 'TARGET']

In [246]:
subm.columns

Index([u'SK_ID_CURR', u'TARGET'], dtype='object')

In [247]:
subm_csv = subm.to_csv('submission.csv', sep=',', index=False)

In [123]:
%cp sample_submission.csv /home/pranay/Downloads/

In [105]:
sample = pd.read_csv('sample_submission.csv', sep=',')

In [104]:
subm.shape

(48744, 2)

In [ ]:
# sample
subm.TARGET.value_counts().plot.bar()

In [58]:
train_y.value_counts()

0    282686
1    24825 
Name: TARGET, dtype: int64

In [187]:
imputed_concatted.shape

(356255, 245)

In [ ]:
get_nulls(concatted_dummied)

In [ ]:
get_nulls(ints)

In [ ]:
objectly = objectly.fillna(objectly.mode())

In [130]:
get_nulls(objectly)

CODE_GENDER                   0.000000 
EMERGENCYSTATE_MODE           47.147128
FLAG_OWN_CAR                  0.000000 
FLAG_OWN_REALTY               0.000000 
FONDKAPREMONT_MODE            68.235393
HOUSETYPE_MODE                49.940632
NAME_CONTRACT_TYPE            0.000000 
NAME_EDUCATION_TYPE           0.000000 
NAME_FAMILY_STATUS            0.000000 
NAME_HOUSING_TYPE             0.000000 
NAME_INCOME_TYPE              0.000000 
NAME_TYPE_SUITE               0.618377 
OCCUPATION_TYPE               31.437032
ORGANIZATION_TYPE             0.000000 
WALLSMATERIAL_MODE            50.591290
WEEKDAY_APPR_PROCESS_START    0.000000 
dtype: float64

In [131]:
objectly.EMERGENCYSTATE_MODE.fillna(objectly.EMERGENCYSTATE_MODE.mode())

0        No 
1        No 
2        NaN
3        NaN
4        NaN
5        NaN
6        NaN
7        NaN
8        NaN
9        NaN
10       NaN
11       NaN
12       No 
13       No 
14       No 
15       NaN
16       NaN
17       NaN
18       No 
19       NaN
20       No 
21       NaN
22       No 
23       No 
24       No 
25       No 
26       Yes
27       NaN
28       NaN
29       No 
         .. 
48714    No 
48715    NaN
48716    No 
48717    NaN
48718    No 
48719    No 
48720    No 
48721    NaN
48722    NaN
48723    NaN
48724    No 
48725    NaN
48726    NaN
48727    No 
48728    NaN
48729    NaN
48730    NaN
48731    No 
48732    No 
48733    No 
48734    NaN
48735    No 
48736    No 
48737    No 
48738    No 
48739    NaN
48740    NaN
48741    No 
48742    No 
48743    NaN
Name: EMERGENCYSTATE_MODE, Length: 356255, dtype: object

In [ ]:
msno.matrix(objectly)

In [ ]:
less_nulls_concat.dtypes.value_counts()

In [ ]:
concatted.loc[:, concatted[less]

In [67]:
assert concatted.shape[0] == train.shape[0] + test.shape[0]

In [ ]:
credit_card_balance.head()

In [45]:
credit_card_balance[credit_card_balance.SK_ID_CURR == 186401].sort_values(by='MONTHS_BALANCE').head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
2019431,1440617,186401,-96,22737.915,112500,0.0,0.0,0.0,0.0,6750.0,96750.0,96750.0,19811.34,22737.915,22737.915,0.0,0,0.0,0.0,15.0,Active,0,0
1476656,2243411,186401,-96,47700.000,180000,45000.0,45000.0,0.0,0.0,NaN,0.0,0.0,45000.00,48996.000,48996.000,2.0,2,0.0,0.0,NaN,Active,0,0
1803594,1440617,186401,-95,30.375,112500,0.0,0.0,0.0,0.0,6750.0,25290.0,25290.0,0.00,0.000,0.000,0.0,0,0.0,0.0,16.0,Active,0,0
2284738,2243411,186401,-95,180277.650,180000,135000.0,135000.0,0.0,0.0,9000.0,9000.0,9000.0,174996.00,178981.650,178981.650,8.0,8,0.0,0.0,1.0,Active,0,0
1473970,2243411,186401,-94,175450.725,180000,0.0,0.0,0.0,0.0,9000.0,9000.0,9000.0,169981.65,175450.725,175450.725,0.0,0,0.0,0.0,2.0,Active,0,0


In [23]:
count_credit = credit_card_balance.groupby('SK_ID_CURR').SK_ID_CURR.count().sort_values(ascending=False).to_frame()

In [ ]:
credit_card_balance.groupby('SK_ID_PREV').SK_ID_PREV.count().sort_values(ascending=False)

In [31]:
count_credit = credit_card_balance.groupby('SK_ID_CURR').SK_ID_CURR.count().sort_values(ascending=False)

In [ ]:
count_credit

In [24]:
count_credit['COUNT'] = count_credit.SK_ID_CURR

In [25]:
count_credit['SK_ID_CURR'] = count_credit.index

In [121]:
merged = pd.merge(train, count_credit, how='inner', on='SK_ID_CURR')

In [26]:
merge_test = pd.merge(test, count_credit, how='inner', on='SK_ID_CURR')

In [ ]:
assert merge_test.shape[0] == test.shape[0]

In [122]:
merged.columns

Index([u'SK_ID_CURR', u'TARGET', u'NAME_CONTRACT_TYPE', u'CODE_GENDER',
       u'FLAG_OWN_CAR', u'FLAG_OWN_REALTY', u'CNT_CHILDREN',
       u'AMT_INCOME_TOTAL', u'AMT_CREDIT', u'AMT_ANNUITY',
       ...
       u'FLAG_DOCUMENT_19', u'FLAG_DOCUMENT_20', u'FLAG_DOCUMENT_21',
       u'AMT_REQ_CREDIT_BUREAU_HOUR', u'AMT_REQ_CREDIT_BUREAU_DAY',
       u'AMT_REQ_CREDIT_BUREAU_WEEK', u'AMT_REQ_CREDIT_BUREAU_MON',
       u'AMT_REQ_CREDIT_BUREAU_QRT', u'AMT_REQ_CREDIT_BUREAU_YEAR', u'COUNT'],
      dtype='object', length=123)

In [123]:
merged.COUNT.isnull().value_counts()

False    86905
Name: COUNT, dtype: int64

In [ ]:
[ col for col in credit_card_balance.SK_ID_CURR.unique() if col not in train.SK_ID_CURR ]

In [11]:
train.TARGET.value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

In [43]:
def get_description_of_columns(df):
    for col in df.columns[2:]:
        print col, descriptions[descriptions.Row == str(col)].Description.values

In [ ]:
get_description_of_columns(credit_card_balance)

In [ ]:
for col in credit_card_balance.columns[2:]:
    print col, descriptions[descriptions.Row == str(col)].Description.values

In [39]:
descriptions.head()

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN


In [ ]:
train.groupby(['TARGET'])['NAME_CONTRACT_TYPE'].value_counts(normalize=True)

In [ ]:
train.groupby(['TARGET'])['CODE_GENDER'].value_counts(normalize=True)

In [ ]:
train.groupby(['TARGET'])['FLAG_OWN_CAR'].value_counts(normalize=True)

In [ ]:
train.groupby(['TARGET'])['FLAG_OWN_REALTY'].value_counts(normalize=True)

In [ ]:
train.groupby(['TARGET'])['CNT_CHILDREN'].value_counts(normalize=True)

In [ ]:
msno.matrix(train[:10000])

In [ ]:
train.head()

In [30]:
train.shape

(307511, 122)

In [58]:
nulls = train.isnull().sum() * 100/train.shape[0]

In [50]:
null_test = test.isnull().sum() * 100/test.shape[0]

In [51]:
above_fifty_null_test = null_test[null_test > 50].index

In [61]:
less_nulls_test = null_test[null_test < 50].index
less_nulls_train = null_test[null_test < 50].index

In [ ]:
msno.matrix(test[less_nulls_test])

In [ ]:
msno.matrix(train[less_nulls_train])

In [52]:
above_fifty_null_test

Index([u'OWN_CAR_AGE', u'BASEMENTAREA_AVG', u'YEARS_BUILD_AVG',
       u'COMMONAREA_AVG', u'ELEVATORS_AVG', u'FLOORSMIN_AVG', u'LANDAREA_AVG',
       u'LIVINGAPARTMENTS_AVG', u'NONLIVINGAPARTMENTS_AVG',
       u'NONLIVINGAREA_AVG', u'BASEMENTAREA_MODE', u'YEARS_BUILD_MODE',
       u'COMMONAREA_MODE', u'ELEVATORS_MODE', u'FLOORSMIN_MODE',
       u'LANDAREA_MODE', u'LIVINGAPARTMENTS_MODE', u'NONLIVINGAPARTMENTS_MODE',
       u'NONLIVINGAREA_MODE', u'BASEMENTAREA_MEDI', u'YEARS_BUILD_MEDI',
       u'COMMONAREA_MEDI', u'ELEVATORS_MEDI', u'FLOORSMIN_MEDI',
       u'LANDAREA_MEDI', u'LIVINGAPARTMENTS_MEDI', u'NONLIVINGAPARTMENTS_MEDI',
       u'NONLIVINGAREA_MEDI', u'FONDKAPREMONT_MODE'],
      dtype='object')

In [ ]:
above_fifty_nulls = nulls[nulls > 50].index
above_fifty_nulls

In [ ]:
[ col for col in above_fifty_nulls if col not in above_fifty_null_test ]

In [57]:
remove_nulls = train.drop(above_fifty_nulls, axis=1)

In [58]:
assert remove_nulls.shape[1] == train.shape[1] - above_fifty_nulls.shape[0]

In [ ]:
msno.matrix(remove_nulls)

In [ ]:
msno.matrix(remove_nulls.iloc[:100, 40:60])

In [ ]:
remove_nulls.isnull().sum() * 100 /(remove_nulls.shape[0])

In [ ]:
for col in remove_nulls.columns[remove_nulls.dtypes == 'object'].values:
    print remove_nulls.groupby(['TARGET'])[str(col)].value_counts(normalize=True)

In [79]:
remove_nulls.dtypes.value_counts()

int64      41
float64    27
object     13
dtype: int64

In [8]:
previous_application.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for col in previous_application.columns.values:
    print col, descriptions[descriptions.Row == str(col)].Description.unique()

In [46]:
previous_application[previous_application.columns[previous_application.dtypes == 'float64'].values].head()

,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,1730.430,17145.0,17145.0,0.0,17145.0,0.0,0.182832,0.867336,12.0,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,25188.615,607500.0,679671.0,NaN,607500.0,NaN,NaN,NaN,36.0,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,15060.735,112500.0,136444.5,NaN,112500.0,NaN,NaN,NaN,12.0,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,47041.335,450000.0,470790.0,NaN,450000.0,NaN,NaN,NaN,12.0,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,31924.395,337500.0,404055.0,NaN,337500.0,NaN,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
previous_application[previous_application.columns[previous_application.dtypes == 'float64']].isnull().sum()

AMT_ANNUITY                  372235 
AMT_APPLICATION              0      
AMT_CREDIT                   1      
AMT_DOWN_PAYMENT             895844 
AMT_GOODS_PRICE              385515 
RATE_DOWN_PAYMENT            895844 
RATE_INTEREST_PRIMARY        1664263
RATE_INTEREST_PRIVILEGED     1664263
CNT_PAYMENT                  372230 
DAYS_FIRST_DRAWING           673065 
DAYS_FIRST_DUE               673065 
DAYS_LAST_DUE_1ST_VERSION    673065 
DAYS_LAST_DUE                673065 
DAYS_TERMINATION             673065 
NFLAG_INSURED_ON_APPROVAL    673065 
dtype: int64

In [60]:
previous_application.SK_ID_PREV.value_counts().head()

1000983    1
2428426    1
1026910    1
1024863    1
2448896    1
Name: SK_ID_PREV, dtype: int64

In [52]:
895844 * 100 / previous_application.shape[0]

53.636480115721696

In [61]:
pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [28]:
previous_application.dtypes.value_counts()

object     16
float64    15
int64      6 
dtype: int64

In [10]:
descriptions.head()

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,"Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)",NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN


In [9]:
descriptions[descriptions.Row == 'AMT_GOODS_PRICE']

,Unnamed: 0,Table,Row,Description,Special
10,13,application_{train|test}.csv,AMT_GOODS_PRICE,For consumer loans it is the price of the goods for which the loan is given,NaN
180,183,previous_application.csv,AMT_GOODS_PRICE,Goods price of good that client asked for (if applicable) on the previous application,NaN


In [ ]:
for col in remove_nulls.columns[remove_nulls.dtypes == 'float64']:
    if (col != 'SK_ID_CURR') and (col != 'TARGET'):
        print remove_nulls.groupby(['TARGET'])[str(col)].mean()

In [ ]:
#getting columns with missing values granular
ranges = [(0,30),(30,60), (60, 120)]

for i in ranges:
    msno.matrix(remove_nulls.iloc[:100, i[0]:i[1]])

In [116]:
remove_nulls.OCCUPATION_TYPE.isnull().value_counts(normalize=True)

False    0.686545
True     0.313455
Name: OCCUPATION_TYPE, dtype: float64

In [129]:
nulls = remove_nulls.isnull().sum() * 100/train.shape[0]

In [136]:
null_cols = nulls[nulls > 30].index.values

In [ ]:
#dtypes of null cols in remove_nulls data frame
for col in null_cols:
    print col, remove_nulls[str(col)].dtype

In [ ]:
#dtypes of missing cols in train
for col in nulls[nulls > 50].index.values:
    print col, train[str(col)].dtypes

In [151]:
pos_cash_balance.isnull().sum()

SK_ID_PREV               0    
SK_ID_CURR               0    
MONTHS_BALANCE           0    
CNT_INSTALMENT           26071
CNT_INSTALMENT_FUTURE    26087
NAME_CONTRACT_STATUS     0    
SK_DPD                   0    
SK_DPD_DEF               0    
dtype: int64

In [147]:
previous_application.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [148]:
previous_application.SK_ID_CURR.value_counts().head()

187868    77
265681    73
173680    72
242412    68
206783    67
Name: SK_ID_CURR, dtype: int64

In [153]:
previous_application.isnull().sum()

SK_ID_PREV                     0      
SK_ID_CURR                     0      
NAME_CONTRACT_TYPE             0      
AMT_ANNUITY                    372235 
AMT_APPLICATION                0      
AMT_CREDIT                     1      
AMT_DOWN_PAYMENT               895844 
AMT_GOODS_PRICE                385515 
WEEKDAY_APPR_PROCESS_START     0      
HOUR_APPR_PROCESS_START        0      
FLAG_LAST_APPL_PER_CONTRACT    0      
NFLAG_LAST_APPL_IN_DAY         0      
RATE_DOWN_PAYMENT              895844 
RATE_INTEREST_PRIMARY          1664263
RATE_INTEREST_PRIVILEGED       1664263
NAME_CASH_LOAN_PURPOSE         0      
NAME_CONTRACT_STATUS           0      
DAYS_DECISION                  0      
NAME_PAYMENT_TYPE              0      
CODE_REJECT_REASON             0      
NAME_TYPE_SUITE                820405 
NAME_CLIENT_TYPE               0      
NAME_GOODS_CATEGORY            0      
NAME_PORTFOLIO                 0      
NAME_PRODUCT_TYPE              0      
CHANNEL_TYPE             

In [152]:
credit_card_balance.isnull().sum()

SK_ID_PREV                    0     
SK_ID_CURR                    0     
MONTHS_BALANCE                0     
AMT_BALANCE                   0     
AMT_CREDIT_LIMIT_ACTUAL       0     
AMT_DRAWINGS_ATM_CURRENT      749816
AMT_DRAWINGS_CURRENT          0     
AMT_DRAWINGS_OTHER_CURRENT    749816
AMT_DRAWINGS_POS_CURRENT      749816
AMT_INST_MIN_REGULARITY       305236
AMT_PAYMENT_CURRENT           767988
AMT_PAYMENT_TOTAL_CURRENT     0     
AMT_RECEIVABLE_PRINCIPAL      0     
AMT_RECIVABLE                 0     
AMT_TOTAL_RECEIVABLE          0     
CNT_DRAWINGS_ATM_CURRENT      749816
CNT_DRAWINGS_CURRENT          0     
CNT_DRAWINGS_OTHER_CURRENT    749816
CNT_DRAWINGS_POS_CURRENT      749816
CNT_INSTALMENT_MATURE_CUM     305236
NAME_CONTRACT_STATUS          0     
SK_DPD                        0     
SK_DPD_DEF                    0     
dtype: int64

In [154]:
credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0
